In [ ]:
import os, sys, h5py, json, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from tqdm import tqdm
    tqdm.pandas() # suppress annoying FutureWarning

import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama import DataGroup, lh5
from pygama.dsp.dsp_optimize import *
from pygama.dsp.WaveformBrowser import WaveformBrowser as wfb

In [ ]:
# config
query = 'cycle > 2185 and cycle < 2188'
tb_in = 'ORSIS3302DecoderForEnergy/raw'
nwfs = 100

In [ ]:
# load data into LH5 Table for DSP
dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
dg.fileDB.query(query, inplace=True)

# limit this test to just the first file in the list
raw_files = dg.lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
raw_file = raw_files.tolist()[0]

# pull a limited number of waveforms passing a cut on energy -- speeds up DSP
ene_vals = lh5.load_nda(raw_file, ['energy'], tb_in)['energy']
ix_ene = np.where((ene_vals > 1e6) & (ene_vals < 1e7))[0][:nwfs]

sto = lh5.Store()
tb_raw, n_rows = sto.read_object(tb_in, raw_file, idx=ix_ene)
# wfs = tb_raw['waveform']['values'].nda

print(ix_ene)
print(n_rows)

In [ ]:
# load config file and run DSP on small table to make sure it doesn't error

dsp_id = '06'
dsp_config = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')

tb_out = run_one_dsp(tb_raw, dsp_config, verbosity=1)

print(tb_out.keys())
df = tb_out.get_dataframe()
df